# Introduction to SolGen 
The SolGen package is a hierarchical simulation tool for photonic circuit. This notebook will cover the basic structure and the usage of the standalone package. Note that the syntax and logic is mainly built to imitate the Nazca Design structure. 

In [3]:
print('Hello Jupyter!')


Hello Jupyter!


In [ ]:
print('Hello Jupyter!')